<a href="https://colab.research.google.com/github/ttbr101081/chatbottelegram/blob/main/Copia_de_ChatBot_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot --upgrade  #instalar python telegram bot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import random
import nltk
import json   #importar libreria para leer archivos json - JavaScript Object Notation
from sklearn.model_selection import train_test_split # división en una parte para entrenar y otra para realizar pruebas
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer #vectorizer
from sklearn.linear_model import LogisticRegression  # clasificador
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def clean(text):
  texto_limpio=''
  for ch in text.lower():
    if ch in 'abcdefghijklmnopqrstuvwxyz':
       texto_limpio = texto_limpio + ch
  return texto_limpio

In [ ]:
with open('/content/BOT_CONFIG.json','r') as f:
   BOT_CONFIG = json.load(f)

#with open('/content/BOT_CONFIG.json','w') as f:     # código para grabar/guardar los cambios realizados en el file json
 #   json.dump(BOT_CONFIG,f)

In [ ]:
# aplicación  de machine learning al chatbot
corpus = []
y = []
for intent in BOT_CONFIG['intents'].keys():
      for example in BOT_CONFIG['intents'][intent]['examples']:
        corpus.append(example)  # texto en examples
        y.append(intent)   # texto de los keys
len(corpus), len(y)  # lista de textos y lista de intents ordenados
# se agrega para mejorar las respuestas del BOT, dividiendo el BOT_CONFIG en una parte para el aprendizaje y otra para testear

(10, 10)

In [ ]:
corpus_train, corpus_test, y_train, y_test = train_test_split( corpus, y, test_size = 0.2, random_state=42)
# --------------------------

#vectorizer = CountVectorizer()
# mejorar el score utilizando las opciones de CountVectorizer()
#vectorizer = CountVectorizer(ngram_range=(1,5))  
# ngram_range permite utilizar combinaciones de palabras no solo palabras
vectorizer = CountVectorizer(ngram_range=(1,5), analyzer= 'char_wb') # analizar por simbolos en vez de por palabras; analyzer
# utilizar otro Vectorizer
#from sklearn.feature_extraction.text import TfidfVectorizer
#vectorizer = TfidfVectorizer(ngram_range = (1,5), analyzer= 'char_wb')
X_train = vectorizer.fit_transform(corpus_train)
X_test = vectorizer.transform(corpus_test)
print(len(vectorizer.get_feature_names_out())) # longitud del diccionario
#vectorizer.get_feature_names_out()
#print(X.toarray())  # imprimir la matriz de coincidencias 

189


In [ ]:
# busqueda de un clasificador para machine learning, p.e.  regresión logistica buscar "LogisticRegression"

#model = LogisticRegression()
# mejorar el modelo de regresión logistica, C=5  obliga aprender más al modelo, 200 iteraciones de aprendizaje
#model= LogisticRegression(C=5, max_iter=200)
# utilizar otro clasificador busqueda "classifier"
model = RandomForestClassifier(n_estimators=300)
model.fit(X_train, y_train) # se entrena el modelo en la parte de entrenamiento (test)
model.score(X_test, y_test) , model.score(X_train, y_train)  # comprobación del modelo utilizando el test y el texto
#model.predict(vectorizer.transform(['que']))  # comprobación que el modelo aprendio


(1.0, 1.0)

In [ ]:
#with open('/content/vectorizer.pickle','wb') as f:
# pickle.dump(vectorizer,f)

In [ ]:
def get_intent(text):
    for intent in BOT_CONFIG['intents'].keys():
      for example in BOT_CONFIG['intents'][intent]['examples']:
        cleaned_example = clean(example) 
        texto_limpio = clean(text)
        if nltk.edit_distance(cleaned_example, texto_limpio) / max(len(cleaned_example),len(texto_limpio)) < 0.4:
          return intent
    return 'not_found :('

# re-escribir la función  get_intent() para que utilice el modelo 
def get_intent_by_model(text):
  return  model.predict(vectorizer.transform([text]))[0]

# ------------------------------------

def bot(text):
  # intent = get_intent(text)   # cambio por el nuevo get_intent_by_model()
  intent = get_intent_by_model(text)
  if intent != 'not_found':
    return random.choice(BOT_CONFIG['intents'][intent]['responses'])
  else:
    return  random.choice(BOT_CONFIG['not_found']['responses'])
 

In [ ]:
# examples/echobot.py
# https://github.com/python-telegram-bot/python-telegram-bot/blob/master/examples/echobot.py

import logging
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    text = update.message.text   #texto entrante desde el bot de telegram
    answer = bot(text)  # respuesta del chatbot
    update.message.reply_text(answer)

    # update.message.reply_text(update.message.text)


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("5558938015:AAHsvrRZht1sx90rZ-OlLBMmtqoHmLTJfvw") # toquen generado por telegram 

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

2022-06-03 01:35:17,307 - apscheduler.scheduler - INFO - Scheduler started
2022-06-03 01:48:07,515 - telegram.ext.updater - ERROR - Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
2022-06-03 01:48:07,519 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/telegram/ext/updater.py", line 646, in _network_loop_retry
    if not action_cb():
  File "/usr/local/lib/python3.7/dist-packages/telegram/ext/updater.py", line 601, in polling_action_cb
    allowed_updates=allowed_updates,
  File "/usr/local/lib/python3.7/dist-packages/telegram/ext/extbot.py", line 232, in get_updates
    api_kwargs=api_kwargs,
  File "/usr/local/lib/python3.7/dist-packages/telegram/bot.py", line 133, in decorator
    result = func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/telegram/bot.py", line 3061,